### Сегментация легких ###

### Анамнез ###
1. В целом, легкие представляют из себя два связных темных пятна на снимках
2. Тем не менее, на ряде изображений этот критерий существенно нарушается: иногда сердце имеет практически такую же проницеамость и отличимо лишь по контурам; иногда в легкие вдаются костные выросты (грудина? искривленный позвоночник), которые на метках исключены. В то же время, например, ключицы входят в область легких, т.к. находятся "перед" ними. Поэтому полагаться только на интенсивность нельзя.
3. Изображений достаточно много, чтобы, с учетом аугментаций, натренировать небольшую сеть.

**Задачу будем решать двумя независимыми способами: при помощи нейросети и классическим алгоритмом**

### Архитектура сети ###

Нейросетевая часть очень похожа на задание 1. Все так же UNet4, по сути, единственный вариант, доступный по вычислительной мощности и способности обучаться на малом наборе данных. 

Для тренировки из изображения вырезается случайный кусок от 0.8 до 1.0 площади исходного изображения, и масштабируется до 128*128. Это несколько насыщает тренировочные данные, не изменяя их типичных особенностей. Например, нерезка на малые куски или отражения наверняка лишь ухудшили бы результат, так как распределение валидационных данных обладало бы общими особенностями (e.g. сердце слева), которыми не обладал бы тренировочный датасет.

Я тренировал сеть при помощи `Adam`, `lr=0.01`, коэффициент затухания `lr=0.9999`, 1 эпоха == длине тренировочного датасета (52 шт), батч = 4 изображения. Результаты проверялись после 100 и 300 эпох тренировки. 300 эпох занимает на моей машине ~45 минут.

### Метрики ###
На мой взгляд, для масок крупных объектов хорошей метрикой для бинарной сегментации является IOU: ее высокие значения обеспечивают, что маски "геометрически" близки. При этом, например, ошибка в виде тонкой линии (которую впоследствии легко исправить пост-процессингом, как в первом задании), внесет существенно меньший вклад, чем круглое пятно (которое, например, на границе областей устранить непросто). Это отвечает геометрической интуиции понятия "хорошего решения".

Также я вычисляю попиксельный F1_score в основном потому, что он остался у меня от первой задачи.и его легко считать. Для бинарной классификации есть и более аккуратные метрики, например коэффициент Мэтьюса. На сколько мне известно, F1 ведет себя *принципиально неадекватно* в случае существенного дисбаланса классов. На картинках, где легкие занимают примерно 1/3 площади и она должна вполне хорошо выражать успешность того или иного алгоритма. Опять же, время не резиновое и часть углов приходится срезать. На практике метрику все равно пришлось бы выбирать прагматически, исходя из имеющейся задачи. Например, если важна мажоранта области, нужно с большим весом учитывать ложно отрицательные пиксели, если миноранта -- то ложно положительные и т.д.
Опять, же достаточно хорошими геометрическими инвариантами являются площадь и периметр.

In [1]:
import data2
import modeltools2
import os
from unet import UNet
import torch, torch.utils, torch.utils.data
from torchsummary import summary
import modeltools1
import datetime
from tqdm.notebook import tqdm
import cv2
import lungsdetector
import random
import numpy as np

data_dir = "../02_image_segmentation2/xray-lung-segmentation"
train_file = "idx-trn0.txt"
val_file = "idx-val0.txt"

# out_dir = "out2"
nn_outdir = "out2/nn"
algo_outdir = "out2/algo"
if not os.path.exists(nn_outdir) or not os.path.exists(algo_outdir): 
    os.makedirs(nn_outdir)
    os.makedirs(algo_outdir)

device = torch.device("cuda")

weights_path = None

# comment all lines below to train the UNet from scratch
# weights after 300 epochs of training
weights_path = "lungs_weights300.dat"

# weights after 100 epochs of training
# weights_path = "lungs_weights100.dat"


In [2]:
train_dataset = data2.LungsDataset(data_dir, train_file)
train_dataset.train()
val_dataset = data2.LungsDataset(data_dir, val_file)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 4, num_workers=2, 
    pin_memory=True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 4, num_workers=2, 
    pin_memory=True)

In [3]:
model = UNet(in_channels = 1, out_channels=2, n_blocks = 4).to(device)
summary(model, (1, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             320
              ReLU-2         [-1, 32, 128, 128]               0
       BatchNorm2d-3         [-1, 32, 128, 128]              64
            Conv2d-4         [-1, 32, 128, 128]           9,248
              ReLU-5         [-1, 32, 128, 128]               0
       BatchNorm2d-6         [-1, 32, 128, 128]              64
         MaxPool2d-7           [-1, 32, 64, 64]               0
         DownBlock-8  [[-1, 32, 64, 64], [-1, 32, 128, 128]]               0
            Conv2d-9           [-1, 64, 64, 64]          18,496
             ReLU-10           [-1, 64, 64, 64]               0
      BatchNorm2d-11           [-1, 64, 64, 64]             128
           Conv2d-12           [-1, 64, 64, 64]          36,928
             ReLU-13           [-1, 64, 64, 64]               0
      BatchNorm2d-14      

In [4]:
lr = 0.01
optim_params = model.parameters()
optim = torch.optim.Adam(optim_params, lr=lr)
loss = modeltools1.FocalLoss(reduction="mean")
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma =0.9999)
epochs = 10 #optimal value is 100

In [5]:
losses = []
if ("weights_path" in locals() or "weights_path" in globals()) and weights_path:
    model.load_state_dict(torch.load(weights_path))
else:
        train_dataset.train()
        model.train()
        modeltools1.train_model(model, loss, optim, scheduler, train_dataloader,
                num_epochs = epochs, device = device) 
        model.cpu()
        current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        torch.save(model.state_dict(), './lungs_weights'+current_time+".dat")

Ниже сетка прогоняется на тренировочном и валидационных датасетах. В папке out2 сохранаются исходное изображение (для наглядности), предсказанная маска и исходная метка.

In [6]:
model.eval()
model.to(device)
train_dataset.eval()
val_dataset.eval()

#note, we output 128*128 images for they are escriptive enough
F1_score, IOU_score = modeltools2.eval_model_on_dataloader(model, train_dataloader, device, f"{nn_outdir}/img_train")
print(f"train F1 score {F1_score}, IOU score: {IOU_score}")   
F1_score, IOU_score = modeltools2.eval_model_on_dataloader(model, val_dataloader, device, f"{nn_outdir}/img_val")
print(f"val F1 score {F1_score}, IOU score: {IOU_score}")   

Validation:   0%|          | 0/53 [00:00<?, ?it/s]

train F1 score 0.9749588588988078, IOU score: 0.9511719683431229


Validation:   0%|          | 0/36 [00:00<?, ?it/s]

val F1 score 0.9585478880011972, IOU score: 0.9266433694832762


### Результаты ###    
    
    Эпох    | F1 train  | IOU train | F1 val    | IOU val  | Loss
    10      | 0.844359  | 0.7507    | 0.820202  | 0.723840 | 0.0147
    100     | 0.968350  | 0.938900  | 0.956945  | 0.923821 | 0.0075
    300     | 0.9749    | 0.9511    | 0.9585    | 0.9266   | 0.0037

Глазометрическое изучение результатов показало, что разница между предсказаниями сетки после 100 и 300 эпох тренировки мало существенна. Из плюсов, это говорит в пользу адекватности наших метрик, они тоже изменились несильно. Из минусов, надежда, что ту же сеть можно натренировать на тех же данных лучше, слабая. Ниже привожу несколько примеров работы сетки (300 эпох) на валидационном датасете.

Зеленым отмечены примеры незначительных ошибок, которые элементарно устраняются пост-процессингом. Красным -- ошибки, которые устранить сложнее. В основном они возникают на снимках с уникальными анатомическими особенностями: сетке просто не хватило таких данных в трейн-датасете. Ну и одно изображение просто супер неконтрастное, а я не делаю никакой входной коррекции, равно как и соответствующих аугментаций. Результат кажется мне достаточно хорошим.
![](lungs_demo1.png)

### Эвристический алгоритм ###

На моей машине инференс проходит со скоростью 8 снимков в секунду вместе с подсчетом метрик и сохранением файлов и в целом сеть дает хороший результат, но эту задачу можно решать и прямым алгоритмом. Это дает больший контроль над морфологией объекта, поэтому в некоторых случах может быть предпочтительным. 

Поскольку в классических алгоритмах обработки изображений я новичок и не претендую на то, чтобы восполнить этот пробел за два дня (хотя в процессе я проникся и таки его восполню), я предъявляю к нему всего два требования:
1. он должен быть параметризован для всего набора данных, то есть я не против подбора некоторых параметров, но они должны подбираться один раз и работать приемлемо на большинстве входных изображений
2. результат должен быть визуально приемлемым. На выходе должно быть ровно две области и IOU с меткой > 0.8

Я попробовал скормить эти картинки ванильному MSER (в реализации opencv 4.5.5), но выявилось несколько проблем: а) он сильно зависит от контрастности изображения. Что гораздо хуже, в силу своей природы, MSER объединяет области в объекты несколькими способами. Так, например, появляются объекты класса легкое+сердце, легкое, легкое без куска и придумать простой алгоритм различения "правильного варианта" мне не удалось. Более того, не видя исходного снимка, я бы и сам не взялся сделать такой выбор. Вот пример объектов, которые MSER находит на одном и том же изоражении. Кроме первого, это все вариации левого легкого.
![](lungs_demo2.png)

Поэтому я решил все же использовать более примитивный алгоритм на основе метода Отсу как начального приближения. После суточного марафона по перестановке кусочков получилось следующее.

1. находим черный цвет на изображении и заменяем его средним по оставшемуся изображению. Это частично решает проблему черной рамки, которая сбивает метод Отсу с толку
2. подкручиваем контраст, обрезая гистограмму (я взял готовую реализацию, источник указан в коде)
3. запускаем метод Отсу, находим начальный threshold по яркости
4. разбираем изображение на компоненты связности, выбираем две с наибольшей площадью
5. если число компонет меньше двух (например, легкие склеились в центре) или если одна из компонент связна с границей изображения, уменьшаем threshold, повторяем последние два шага
6. используем замыкание и раздутие, чтобы получить более гладкую маску.

Следующий фрагмент выбирает случайное изображение из тестового набора и сохраняет для него все шаги алгоритма в папку `algo_outdir`.

In [7]:
i = random.randint(0, len(train_dataset) - 1)
train_dataset.numpy()
img, lbl = train_dataset[i]
img = lungsdetector.channels_last(img)
lbl = lungsdetector.channels_last(lbl)

In [8]:
rslt = lungsdetector.separate_lungs(img, 10, 40, 25, 3, steps_file = f'{algo_outdir}/rnd')
if rslt is not None:
    rslt_filt = lungsdetector.smoothen(rslt, 10, 2) 
    cv2.imwrite(f'{algo_outdir}/rnd_filtered_result.png', rslt_filt)
    cv2.imwrite(f'{algo_outdir}/rnd_filtered_label.png', ((lbl > 0) *255).astype(np.uint8))
    f1 = lungsdetector.F1_single_input(rslt_filt, lbl)
    iou = lungsdetector.IOU_single_metrics(rslt_filt, lbl)
    print(f1,iou)

0.8858357105431681 0.7950674


### Пример работы ###
![](lungs_demo3.png)

Прогоним метрики

In [9]:
train_dataset.numpy()
val_dataset.numpy()

#F1 and IOU are computed only for successfully segmented images
f1, iou, failed = lungsdetector.eval_on_dataset(train_dataset, otsu_step = 2, 
    black_limit = 40, clip_percent = 25, border_thickness = 3, closing_rad = 10, dilation_rad = 2,
    max_steps = 60, out_file = f'{algo_outdir}/test', verbose = False)
print(f"train F1 score {f1}, IOU score: {iou}, failed to classify: {failed}, failure rate: {failed / len(train_dataset)}") 

f1, iou, failed = lungsdetector.eval_on_dataset(val_dataset, otsu_step = 2, 
    black_limit = 40, clip_percent = 25, border_thickness = 3, closing_rad = 10, dilation_rad = 2,
    max_steps = 60, out_file = f'{algo_outdir}/test', verbose = False)
print(f"val F1 score {f1}, IOU score: {iou}, failed to classify: {failed}, failure rate: {failed / len(val_dataset)}")   

Validation:   0%|          | 0/212 [00:00<?, ?it/s]

Can't separate image: 60
Can't separate image: 65
Can't separate image: 67
Can't separate image: 87
Can't separate image: 124
Can't separate image: 153
Can't separate image: 168
train F1 score 0.8501824767535043, IOU score: 0.7524567438334954, failed to classify: 7, failure rate: 0.0330188679245283


Validation:   0%|          | 0/142 [00:00<?, ?it/s]

Can't separate image: 20
Can't separate image: 35
Can't separate image: 48
Can't separate image: 51
Can't separate image: 52
Can't separate image: 56
Can't separate image: 68
Can't separate image: 125
Can't separate image: 127
val F1 score 0.8605183551878992, IOU score: 0.7639141593660627, failed to classify: 9, failure rate: 0.06338028169014084


### Выводы ###
В целом, не могу сказать, что алгоритм получился  удачным, ни с точки зрения метрик, ни при визуальной оценке. Заявленное условие IOU>0.8 (которе UNet достигает чуть и не за 10 эпох обучения) в среднем, увы, не достигнуто. Частые проблемы: объединение областей в результате финальной фильтрации, либо наоборот чрезмерная редукция в попытке отделиться от границы изображения. При этом увеличение шага threshold (`otsu_step`) приводит к улучшению по первому пункту и ухудшению по второму, а уменьшение радиуса раздутия (`dilation_rad`), наоборот. В целом сетевая реализация работает гораздо лучше.

Подбор параметров в качестве ablation study показал, что все части алгоритма существенны, но детальный анализ выходит за рамки моих возможностей по времени.

**Пример работы:**

![](lungs_demo4.png)

### Что не получилось ###
Изначально я намеревался применять метод Отсу один раз, а склеившиеся объекты (как правило, легкое и мусор у границы) разделять watershedding'ом, но не смог правильно вычислить начальные маркеры: ни точки минимума градиента (что должно отвечать "однородно залитым областям"), ни точки максимума расстояний от границы не подошли. Таких точек слишком много и watershedding выдает лоскутное одеяло. 